<h1 align="center">MLB War Predictor</h1>


## Plan

1. Web scape player stats for features
2. Add them to a SQL database 
3. Use SQL to make the features ready for ML
4. Pull the features into a pandas dataframe
5. Fill in any missing stats and split into train test splits
6. Train and evaluate the Pytorch model
7. Send the results back into SQL
8. USe the SQL db to make a PowerBI dashboard

#### Add Ons
1. Contract efficiency values for the dashboard (Maybe extend into a contract predictor)
2. Add in an adjustment to WAR using injury prediction


## Note

As this is all web scraped from fangraphs as it does not have dynamic content I will be using fwar as my statistic.  
I am also doing only hitters to start as pitchers would require a new framework 

# Step 1.

In [1]:
import pandas as pd
from pybaseball import batting_stats_range, statcast_batter
import os

In [5]:
startingYear = 2023
endingYear = 2025

stats = pd.DataFrame()

for year in range(startingYear, endingYear + 1):
    batStats = batting_stats_range(start_dt=f"{year}-01-01", end_dt=f"{year}-12-01")
    batStats['season'] = year
    stats = pd.concat([stats, batStats], ignore_index=True)

IndexError: list index out of range